### Loading the global DEM

In [14]:
import geopandas as gpd
import numpy as np 
from scipy.interpolate import griddata
from scipy.interpolate import Rbf
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 17})  
import pyshtools as pysh


#Loading files
base_layer = gpd.read_file(r"C:\Users\DELL\Downloads\Volume and Interpolation _Project_layers\elevations_2\base elevations_2.shp")
raised_layer = gpd.read_file(r"C:\Users\DELL\Downloads\Volume and Interpolation _Project_layers\elevations_2\raised elevations_2.shp") 
#FROM ATTRIBUTE TABLE
x_b = base_layer['X'].values
y_b = base_layer['Y'].values
z_b = base_layer['elevations'].values

#-------Load MOLA spherical harmonic coefficients---------------# 
mola_sh = pysh.datasets.Mars.MOLA_shape()    #Use lmax = 3000 in brackets for to download the entire dataset with 3000 degree, without this th elater expansion will stretch the aailable data hecne the pixelised appearance.
mola_sh.coeffs[0,0,0] = 0 #mean planetary radius
mola_sh.coeffs[0,2,0] = 0 #C20 component 
#expansion: transforming from spherical harmonic coefficients into a spatial grid representation so ---> this expands the data from spherical harmonic space to physical space (a global grid of elevation values).
mola_grid_obj = mola_sh.expand(grid='DH', lmax=  3000)  
mola_grid = mola_grid_obj.data                       
np.save('MOLA_GLobalGrid_saved_lmax%i_lmax_calc200'%(mola_sh.lmax), mola_grid)

#We know the Driscoll-Healy (DH) grid spans: #Latitude: from 90° (North Pole) to -90° (South Pole) #longitude 0° to 360° 
# we can define it to be able to parse desired extent later

lat_mola = mola_grid_obj.lats() #np.linspace (90, -90, n_lat)
lon_mola = mola_grid_obj.lons() #np.linspace (0, 360, n_lon, endpoint= False)

# print key details
print("Max degree available:", mola_sh.lmax)
print("Global MOLA grid shape:", mola_grid.shape)
 
# we can either plot the global grid using built-in method
#mola_grid_obj.plot(cmap="terrain", show=True)
#fig.show()

# or since we have the array representing all the elevation values that is mola_grid = mola_grid_obj.data
#we can directly plot the contour map 

Max degree available: 719
Global MOLA grid shape: (6003, 6003)


### 1. Plotting the Noctis with a contextual Global MOLA DEM using planetary data modules from pyshtools library
- mola_sh.coeffs [0,0,0] = 0 is equivalent to making the martian geoid (Aeroid) equal to zero.
    -- When you remove the a0,0 term (the mean), each elevation value is then expressed as a deviation from the global average. In other words, instead of saying "this point is 3403 km from the center of Mars," you're saying "this point is 7 km above the average surface height" (assuming 3403 km was 7 km above the mean, for example).
- The overall dataset now has a mean of 0 m. 


In [15]:
%matplotlib qt

import matplotlib.patches as patches
import matplotlib.pyplot as plt

# ----> Desired extent (for noctis mons)-------#
lon_min, lon_max = np.min(x_b)+ 360, np.max(x_b)+ 360  # (convert longitude to 0–360 range fro DH grid) 
lat_min, lat_max = np.min(y_b), np.max(y_b) #already recognized in DH from -90 S to +90 N    
lon_inds = np.where((lon_mola >= lon_min) & (lon_mola <= lon_max))[0] #this parses first element from lon_mola 
lat_inds = np.where((lat_mola >= lat_min) & (lat_mola <= lat_max))[0]

# extract the grid from the mola grid
grid_noctis = mola_grid[np.ix_(lat_inds, lon_inds)]
np.save('grid_noctis_saved_lmax%i'%(mola_sh.lmax), grid_noctis)
#--------#----------------#

#----------------PLotting-----------#
# single figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(15,10))
#1 Global MOLA Grid #
ax1 = axes[0]
im1 = ax1.imshow(mola_grid, cmap="terrain", origin="upper",
                 extent=[np.min(lon_mola), np.max(lon_mola), np.min(lat_mola), np.max(lat_mola)])

#zoomed-in on desired region
ax1.set_xlim(225,325)
ax1.set_ylim(-30,30)

#------------------vectors to annotate------------#
polygon_vertices = [
    (lon_min, lat_min),  # Bottom-left
    (lon_max, lat_min),  # Bottom-right
    (lon_max, lat_max),  # Top-right
    (lon_min, lat_max),  # Top-left
    (lon_min, lat_min)   # Closing the loop
]
polygon = patches.Polygon(polygon_vertices, edgecolor = 'red',facecolor = 'none', closed=True, linewidth = 2.0)
arrow = patches.FancyArrow(-7, 265, 5, -10, color = 'blue', width=1.0,lw=5)   

# the vectors must appear on the plot #if we'd used ax1.annotate no need to add_patch since it is already called by ax1
ax1.add_patch(polygon)
ax1.add_patch (arrow)
#-------------------##-----------#
# Titles and labels
ax1.set_title("Global Mars MOLA Topography (for context)")
ax1.set_xlabel("Longitude")
ax1.set_ylabel("Latitude")
# colorbar
cbar1 = fig.colorbar(im1, ax=ax1, orientation = 'horizontal')
cbar1.set_label("Elevation (m)",)

#2-----Noctis Mons plot-------#
ax2 = axes[1]
# Plot the extracted subgrid
im2 = ax2.imshow(grid_noctis, origin='upper', cmap='terrain',
                 extent=[lon_min, lon_max, lat_min, lat_max])
# Titles and labels
ax2.set_title("Extracted MOLA Grid for defined ROI (Present Day)")
ax2.set_xlabel("Longitude")
ax2.set_ylabel("Latitude")
#colorbar
ax2.tick_interval=[45, 30]
cbar2 = fig.colorbar(im2, ax=ax2, orientation = 'horizontal')
#ax2.contour(grid_x_b, grid_y_b, grid_noctis[::-1],levels= 4, colors = 'k', origin = 'upper')
cbar2.set_label("Elevation (m)")
# Adjust layout
plt.show()
# 1) Save the figure to disk (PNG at 300 dpi):
plt.savefig("noctis_mons_extract.png", dpi=300)


np.save('grid_noctis.npy', grid_noctis)


In [24]:
import notebook
print(notebook.__version__)


7.4.2


## 2. Interpolation for Base level

In [17]:
plt.rcParams.update({'font.size': 26})


base_layer = gpd.read_file(r"C:\Users\DELL\Downloads\Volume and Interpolation _Project_layers\elevations_2\base elevations_2.shp")
raised_layer = gpd.read_file(r"C:\Users\DELL\Downloads\Volume and Interpolation _Project_layers\elevations_2\raised elevations_2.shp") 

#.values attribute returns a NumPy array containing just the values without any associated index or column information from the DataFrame or Series.
#essentially without the values attribute we will be extracting the all the associated informaiton in th table like column, data type etc

x_b = base_layer['X'].values +360
y_b = base_layer['Y'].values
z_b = base_layer['elevations'].values

#--------------------Interpolation------------# 
#radial basis function we would use TPS for a continuous smooth volcano shape
#but we want normal true interpolation with least exaggeration ((see TPS image))

# now we will create an empty grid with resolution maching the present day DEM grid generated above)
grid_x_b, grid_y_b = np.meshgrid (
    np.linspace(min(x_b), max(x_b), np.shape(grid_noctis)[1]),   
    np.linspace(min(y_b), max(y_b), np.shape(grid_noctis)[0])
)

# Radial Basis Function (RBF) interpolation (Spline)
rbf_interpolator = Rbf(x_b, y_b, z_b, function='linear') #-->input is known coordinates, known elevation
grid_z_spline_b = rbf_interpolator(grid_x_b, grid_y_b) #input required coordinates, outputs the required elevation 


#---------------Plotting---------------------#

# single figure with two subplots
fig, ax2 = plt.subplots(1)  

#1 Global MOLA Grid #
ax1 = axes[0]
im1 = ax1.imshow(mola_grid, cmap="terrain", origin="upper",
                 extent=[np.min(lon_mola), np.max(lon_mola), np.min(lat_mola), np.max(lat_mola)])

#zoomed-in on desired region
ax1.set_xlim(225,325)
ax1.set_ylim(-30,30)

#-vectors to annotate-#
polygon_vertices = [
    (lon_min, lat_min),  # Bottom-left
    (lon_max, lat_min),  # Bottom-right
    (lon_max, lat_max),  # Top-right
    (lon_min, lat_max),  # Top-left
    (lon_min, lat_min)]   # Closing the loop
polygon = patches.Polygon(polygon_vertices, edgecolor = 'red',facecolor = 'none', closed=True, linewidth = 2.0)
arrow = patches.FancyArrow(-7, 265, 5, -10, color = 'blue', width=1.0,lw=5)   
# the vectors must appear on the plot #if we'd used ax1.annotate no need to add_patch since it is already called by ax1
ax1.add_patch(polygon)
ax1.add_patch (arrow)
# Titles and labels

ax1.set_title("Global Mars MOLA Topography (for context)")
ax1.set_xlabel("Longitude")
ax1.set_ylabel("Latitude")
# colorbar
#cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label("Elevation (m)")

#------- interpolation plot -----------#
#ax2 = axes[1]
#contourf means contour filled with all the colors, parameters etc, contour would then mean just the contour lines
contour = ax2.contourf(grid_x_b, grid_y_b, grid_z_spline_b[::-1], cmap='terrain', levels=50,  #levels are like how many contour lines would appear
                    origin = 'upper')
#no need to assign a variable unless you wanna use it later in the code, python has problem with grid so it onverses itself so we had to reinverse it for plotting
ax2.contour(grid_x_b, grid_y_b, grid_noctis[::-1],levels= 3, colors = 'k', origin = 'upper')

#ax2.pcolormesh(grid_x_b, grid_y_b, grid_noctis)

# Scatter plot for input points
scatter = ax2.scatter(x_b, y_b, c='magenta', s=30, label = 'base elevations (QGIS)')

# Color bar
cbar = fig.colorbar(contour, ax=ax2)
cbar.set_label("Elevation (m)")


# Titles and labels
ax2.set_title("Base Surface Interpolated Grid (Spline Interpolation)")
ax2.set_xlabel("Longitude")
ax2.set_ylabel("Latitude")

plt.tight_layout()  # adjusts subplots to fit
plt.legend()
plt.show()

# Print the min and max values for base elevations
print(f'Actual minimum and maximum values of interpolated base are: {np.min(grid_z_spline_b)} and {np.max(grid_z_spline_b)} respectively')


Actual minimum and maximum values of interpolated base are: -588.7934834684074 and 6662.00476056989 respectively


Explanantion cell

### Spherical Harmonics:  f(θ,ϕ)=∑(l=0 to ∞) ∑ (m= −l to l)) a_l,m Y(l to m) (θ,ϕ)

    - Y(l to m) (θ,ϕ) are spherical harmonic functions 
    - a_l,m are coefficients that tells you how much of each spherical harmonics contribute to f(θ,ϕ)
    
- In 2001, MOLA data --> newly defined aeroid: the equipotential surface (gravitational plus rotational) whose average value at the equator is equal to the mean radius of the planet.
- Normally, DEM values means how far the surface is above or below that defined reference level (e.g., a sphere of radius equal to Mars’s mean radius= Aeroid).

- a 0,0 --> determines the global average (or mean) of the elevation over the entire sphere. It tells you what the “baseline” or overall offset is.



## 3*. Calculating the thickness grid and plotting it : (this is present day thickness as opposed to past thickness which is uplift_grid in second notebook)
- Subtract grid_z_spline_b from mola_grid --->height of the exisiting raised features
- we had to change the grid_z_spline_b grid resolution to match the one of the extent above

In [5]:

difference_grid = (grid_noctis[::-1]) - (grid_z_spline_b)  # re-reversing rows for contour lines

# Create the filled contour plot and capture its mappable object.
contourf_obj = plt.contourf(grid_x_b, grid_y_b, difference_grid, cmap='terrain', levels=50, origin='upper')

# Plot the radargram profile line.
data = np.load('line_data.npz')
longitudes_tab = data['longitudes']
latitudes_tab = data['latitudes']
plt.legend()
# Attach a colorbar only to the contourf object.
plt.colorbar(contourf_obj, label='uplift (m) (present day - low points)')

# Plot a scatter if needed.
plt.scatter(x_b, y_b, c='brown', s=10)

print(f'{np.max(difference_grid)/1000} kms')

# Add contour lines for geological context.
plt.contour(grid_x_b, grid_y_b, grid_noctis[::-1], levels=2, colors='k', origin='upper')
plt.xlim(np.min(grid_x_b), np.max(grid_x_b))
plt.ylim(np.min(grid_y_b), np.max(grid_y_b))
plt.title('Thickness grid \n (Current topography - interpolated (exposed) base)')
plt.xlabel('Longitudes')
plt.ylabel('Latitudes')
plt.show()

np.save('difference_grid.npy', difference_grid)


6.070765956647162 kms


In [6]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(grid_x_b, grid_y_b, difference_grid, cmap='viridis')
ax.contour(grid_x_b, grid_y_b, grid_noctis[::-1], levels=2, colors='k', origin='upper')
ax.scatter(x_b, y_b, c='brown', s=10)
ax.set_title('3D Surface Plot of Thickness')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Thickness (m)')
plt.show()


## 2. Interpolation for Raised level


In [12]:
x = raised_layer['X'].values+360
y = raised_layer['Y'].values
z = raised_layer['elevations'].values

#---------Interpolation--------#
grid_x, grid_y = np.meshgrid(
    np.linspace(min(x), max(x), np.shape(grid_noctis)[1]),       
    np.linspace(min(y), max(y), np.shape(grid_noctis)[0]))

# Radial Basis Function (RBF) interpolation (Spline)
rbf_interpolator = Rbf(x, y, z, function='linear')
grid_z_spline_r = rbf_interpolator(grid_x, grid_y)

#-------------------Plotting------------------#

# single figure with two subplots
fig, ax2 = plt.subplots(1)  

#1 Global MOLA Grid #
ax1 = axes[0]
im1 = ax1.imshow(mola_grid, cmap="terrain", origin="upper",
                 extent=[np.min(lon_mola), np.max(lon_mola), np.min(lat_mola), np.max(lat_mola)])
#zoomed-in on desired region
ax1.set_xlim(225,325)
ax1.set_ylim(-30,30)
#-vectors to annotate-#
polygon_vertices = [
    (lon_min, lat_min),  # Bottom-left
    (lon_max, lat_min),  # Bottom-right
    (lon_max, lat_max),  # Top-right
    (lon_min, lat_max),  # Top-left
    (lon_min, lat_min)]   # Closing the loop
polygon = patches.Polygon(polygon_vertices, edgecolor = 'red',facecolor = 'none', closed=True, linewidth = 2.0)
arrow = patches.FancyArrow(-7, 265, 5, -10, color = 'blue', width=1.0,lw=5)   
# the vectors must appear on the plot #if we'd used ax1.annotate no need to add_patch since it is already called by ax1
ax1.add_patch(polygon)
ax1.add_patch (arrow)
# Titles and labels
ax1.set_title("Global Mars MOLA Topography (for context)")
ax1.set_xlabel("Longitude")
ax1.set_ylabel("Latitude")
# colorbar
#cbar1 = fig.colorbar(im1, ax=ax1, shrink = 0.6)
#cbar1.set_label("Elevation (m)")

#------- interpolation plot -----------#
#ax2 = axes[1]

# Contour plot for the interpolated surface
contour = ax2.contourf(grid_x, grid_y, grid_z_spline_r, cmap='terrain', levels=50)
#geological context
ax2.contour(grid_x, grid_y, grid_noctis[::-1],levels= 4, colors = 'k', origin = 'upper') #try using  grid_noctis[::-1] as matrix might be reversed

# Scatter plot for input points
scatter = ax2.scatter(x, y, c='orange', s=20, label="raised samples")


cbar = fig.colorbar(contour, ax=ax2)
cbar.set_label("Elevation (m)")

# Titles and labels
ax2.set_title("Interpolated raised/high surface (spline)")
ax2.set_xlabel("Longitude(°)")
ax2.set_ylabel("Latitude(°)")

# Show the plot
plt.tight_layout() 
plt.legend(loc = 'lower left')
plt.show()






 # using the current day topography and adding the rise interpolation?

In [29]:
addition = (grid_noctis)+(grid_z_spline_r)
print (f'{np.max(addition)/1000} kms')

18.51915754588709 kms


In [30]:
print (grid_z_spline_r.shape)
print ()

(160, 89)



In [31]:
mola_sh = pysh.datasets.Mars.MOLA_shape(lmax=1000)    #Use lmax = 3000 in brackets for to download the entire dataset with 3000 degree, without this th elater expansion will stretch the aailable data hecne the pixelised appearance.
mola_sh.coeffs[0,0,0] = 0
mola_sh.coeffs[0,2,0] = 0
#expansion: transforming from spherical harmonic coefficients into a spatial grid representation so ---> this expands the data from spherical harmonic space to physical space (a global grid of elevation values).
mola_grid_obj = mola_sh.expand(grid='DH')  
mola_grid = mola_grid_obj.data                       
np.save('MOLA_GLobalGrid_saved_lmax%i'%(mola_sh.lmax), mola_grid)
print (mola_sh.lmax)

1000


In [32]:
print (np.mean([3.12,3.5]))

3.31


In [55]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import geopandas as gpd
from scipy.interpolate import Rbf
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize



plt.rcParams.update({'font.size': 20})

#---------------------- Load Data ------------------------#
base_layer = gpd.read_file(r"C:\Users\DELL\Downloads\Volume and Interpolation _Project_layers\elevations_2\base elevations_2.shp")
raised_layer = gpd.read_file(r"C:\Users\DELL\Downloads\Volume and Interpolation _Project_layers\elevations_2\raised elevations_2.shp") 

x_b, y_b, z_b = base_layer['X'].values + 360, base_layer['Y'].values, base_layer['elevations'].values
x_r, y_r, z_r = raised_layer['X'].values + 360, raised_layer['Y'].values, raised_layer['elevations'].values

# Assume grid_noctis, mola_grid, lon_mola, lat_mola, lon_min, lon_max, lat_min, lat_max already exist

# Grid for interpolation
grid_x, grid_y = np.meshgrid(
    np.linspace(min(min(x_b), min(x_r)), max(max(x_b), max(x_r)), np.shape(grid_noctis)[1]),
    np.linspace(min(min(y_b), min(y_r)), max(max(y_b), max(y_r)), np.shape(grid_noctis)[0])
)

# Interpolate both surfaces
rbf_base = Rbf(x_b, y_b, z_b, function='linear')
grid_z_base = rbf_base(grid_x, grid_y)

rbf_raised = Rbf(x_r, y_r, z_r, function='linear')
grid_z_raised = rbf_raised(grid_x, grid_y)

# Shared color range for both plots
vmin = min(grid_z_base.min(), grid_z_raised.min())
vmax = max(grid_z_base.max(), grid_z_raised.max())



# create a “blank” mappable with the shared colormap and global norm
norm = Normalize(vmin=vmin, vmax=vmax)
sm   = ScalarMappable(norm=norm, cmap='terrain')
sm.set_array([])   # no actual data needed

#---------------------- Plotting ------------------------#
fig, (ax2, ax1) = plt.subplots(1, 2, constrained_layout=True)

# Base Surface Plot
contour1 = ax1.contourf(grid_x, grid_y, grid_z_base[::-1], cmap='terrain', levels=50, vmin=vmin, vmax=vmax)
ax1.contour(grid_x, grid_y, grid_noctis[::-1], levels=4, colors='k', origin='upper')
ax1.scatter(x_b, y_b, c='magenta', s=30, label='Base samples')
ax1.set_title("Base Surface Interpolation")
ax1.set_xlabel("Longitude")
ax1.set_ylabel("Latitude")

# Raised Surface Plot
contour2 = ax2.contourf(grid_x, grid_y, grid_z_raised[::-1], cmap='terrain', levels=50, vmin=vmin, vmax=vmax)
ax2.contour(grid_x, grid_y, grid_noctis[::-1], levels=4, colors='k', origin='upper')
ax2.scatter(x_r, y_r, c='orange', s=30, label='Raised samples')
ax2.set_title("Raised Surface Interpolation")
ax2.set_xlabel("Longitude")
ax2.set_ylabel("Latitude")

# then later, after plotting your two subplots…
cbar = fig.colorbar(sm, ax=[ax1, ax2], orientation='vertical', pad=0.01, fraction = 0.09)    # thickness: 5% of the figure height)
cbar.set_label("Elevation (m)")

# Show legends
ax1.legend(loc='lower right')
ax2.legend(loc='lower right')
plt.show()

# Print range info
print(f"Interpolated Base Elevation: min={grid_z_base.min():.2f}, max={grid_z_base.max():.2f}")
print(f"Interpolated Raised Elevation: min={grid_z_raised.min():.2f}, max={grid_z_raised.max():.2f}")


Interpolated Base Elevation: min=-585.07, max=6694.15
Interpolated Raised Elevation: min=3143.94, max=8942.65


C:\Users\DELL\anaconda3\envs\dlrinternship\Lib\site-packages\ipykernel\eventloops.py:145: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  el.exec() if hasattr(el, "exec") else el.exec_()
